# Update `metadata_prod` table using the data in `metadata_staging table`
With this notebook we can update the metadata_prod table (which contains the metadata that we use on the Half-Earth Map) with the data in the metadata_staging table (in which all the new metadata is tested before sending it to production)


### Import packages

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os

### Connect to ESRI

In [2]:
env_path = ".env" # set here the path to your .env file
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [3]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [4]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


### Call the tables
**Metadata_prod**

In [5]:
item_prod = gis.content.get('cab2acd857a34e2faef1f60a9d40e354')
flayer_prod = item_prod.tables[0]
prod_fset = flayer_prod.query() #querying without any conditions returns all the features
prod_sdf = prod_fset.sdf
prod_sdf.head()

,layerSlug,description,source,molLogo,hasAdditionalContent,title,ObjectId,GlobalID,ObjectId2
0,urban_human_pressures,Shows areas where the land is used by urban ac...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Urban pressures metadata,None,136e5a63-a0e7-4100-9741-2650b353e36e,1
1,irrigated_human_pressures,Shows areas where the land is used by irrigate...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Irrigated agriculture metadata,None,0e9c36e0-fa1d-4f76-9ad2-a2beccdb18f0,2
2,rainfed_human_pressures,Shows areas where the land is used by rainfed ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rainfed agriculture metadata,None,a7fbc232-6689-4a65-939a-089f7226e527,3
3,rangeland_human_pressures,Shows areas where the land is used by rangelan...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rangeland metadata,None,84c1634d-9ce7-4e8e-bfbc-f44ce852c4cf,4
4,merged_land_human_pressures,Shows areas where there is high anthropogenic ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Land human pressures metadata,None,4d862350-7f89-4c5c-8b4b-cb1271e66b1e,5


In [6]:
prod_sdf.tail()

,layerSlug,description,source,molLogo,hasAdditionalContent,title,ObjectId,GlobalID,ObjectId2
74,butterflies-richness,Species richness at the country level for butt...,"(1) [Pinkert S. et al., 2022](https://onlineli...",TRUE,FALSE,Butterflies richness by country metadata,None,None,75
75,odonates-richness,Species richness at the country level for odon...,"(1) Sandall, E.L., Pinkert S., & Jetz, W. (20...",TRUE,FALSE,Odonates richness by country metadata,None,None,76
76,sapindales-richness,Species richness at the country level for sapi...,(1) [Plants of the World Online (2019). Facili...,TRUE,FALSE,Sapindales richness by country metadata,None,None,77
77,butterflies-richness-1km,Each cell in this view measures 1 km x 1 km. W...,Map of Life and supporting datasets.,TRUE,FALSE,Butterflies richness metadata,None,136e5a63-a0e7-4100-9741-2650b353e36e,78
78,butterflies-rarity-1km,Each cell in this view measures 1 km x 1 km. W...,Map of Life and supporting datasets.,TRUE,FALSE,Butterflies rarity metadata,None,136e5a63-a0e7-4100-9741-2650b353e36e,79


**Metadata_staging**

In [7]:
item_stag = gis.content.get('ef369a73779d4a37b2252808afef98a7')
flayer_stag = item_stag.tables[0]
stag_fset = flayer_stag.query() #querying without any conditions returns all the features
stag_sdf = stag_fset.sdf
stag_sdf.head()

,layerSlug,description,source,molLogo,hasAdditionalContent,title,ObjectId,GlobalID,ObjectId2,ObjectId3
0,urban_human_pressures,Shows areas where the land is used by urban ac...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Urban pressures metadata,None,136e5a63-a0e7-4100-9741-2650b353e36e,1,1
1,irrigated_human_pressures,Shows areas where the land is used by irrigate...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Irrigated agriculture metadata,None,0e9c36e0-fa1d-4f76-9ad2-a2beccdb18f0,2,2
2,rainfed_human_pressures,Shows areas where the land is used by rainfed ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rainfed agriculture metadata,None,a7fbc232-6689-4a65-939a-089f7226e527,3,3
3,rangeland_human_pressures,Shows areas where the land is used by rangelan...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rangeland metadata,None,84c1634d-9ce7-4e8e-bfbc-f44ce852c4cf,4,4
4,merged_land_human_pressures,Shows areas where there is high anthropogenic ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Land human pressures metadata,None,4d862350-7f89-4c5c-8b4b-cb1271e66b1e,5,5


In [8]:
stag_sdf.tail()

,layerSlug,description,source,molLogo,hasAdditionalContent,title,ObjectId,GlobalID,ObjectId2,ObjectId3
81,mammals-rarity-1km,Each cell in this view measures 1 km x 1 km. W...,Map of Life and supporting datasets.,TRUE,FALSE,Mammals rarity metadata,None,None,82,82
82,summer-birds-richness-1km,Breeding ranges of migratory birds. Each cell ...,Map of Life and supporting datasets.,TRUE,FALSE,Summer birds richness metadata,None,None,83,83
83,summer-birds-rarity-1km,Breeding ranges of migratory birds. Each cell ...,Map of Life and supporting datasets.,TRUE,FALSE,Summer birds rarity metadata,None,None,84,84
84,winter-birds-richness-1km,Non-breeding ranges of migratory birds. Each c...,Map of Life and supporting datasets.,TRUE,FALSE,Winter birds richness metadata,None,None,85,85
85,winter-birds-rarity-1km,Non-breeding ranges of migratory birds. Each c...,Map of Life and supporting datasets.,TRUE,FALSE,Winter birds rarity metadata,None,None,86,86


### Identify mismatches between these two tables
Source [notebook](https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/) by ESRI 

#### 1. Identify fields that have different values

In [9]:
## Columns that we want to update
updatable_cols = ['layerSlug',
 'description',
 'source',
 'molLogo',
 'hasAdditionalContent',
 'title',
 'ObjectId2']

In [10]:
prod_sdf[updatable_cols]

,layerSlug,description,source,molLogo,hasAdditionalContent,title,ObjectId2
0,urban_human_pressures,Shows areas where the land is used by urban ac...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Urban pressures metadata,1
1,irrigated_human_pressures,Shows areas where the land is used by irrigate...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Irrigated agriculture metadata,2
2,rainfed_human_pressures,Shows areas where the land is used by rainfed ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rainfed agriculture metadata,3
3,rangeland_human_pressures,Shows areas where the land is used by rangelan...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rangeland metadata,4
4,merged_land_human_pressures,Shows areas where there is high anthropogenic ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Land human pressures metadata,5
...,...,...,...,...,...,...,...
74,butterflies-richness,Species richness at the country level for butt...,"(1) [Pinkert S. et al., 2022](https://onlineli...",TRUE,FALSE,Butterflies richness by country metadata,75
75,odonates-richness,Species richness at the country level for odon...,"(1) Sandall, E.L., Pinkert S., & Jetz, W. (20...",TRUE,FALSE,Odonates richness by country metadata,76
76,sapindales-richness,Species richness at the country level for sapi...,(1) [Plants of the World Online (2019). Facili...,TRUE,FALSE,Sapindales richness by country metadata,77
77,butterflies-richness-1km,Each cell in this view measures 1 km x 1 km. W...,Map of Life and supporting datasets.,TRUE,FALSE,Butterflies richness metadata,78


In [11]:
stag_sdf[updatable_cols]

,layerSlug,description,source,molLogo,hasAdditionalContent,title,ObjectId2
0,urban_human_pressures,Shows areas where the land is used by urban ac...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Urban pressures metadata,1
1,irrigated_human_pressures,Shows areas where the land is used by irrigate...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Irrigated agriculture metadata,2
2,rainfed_human_pressures,Shows areas where the land is used by rainfed ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rainfed agriculture metadata,3
3,rangeland_human_pressures,Shows areas where the land is used by rangelan...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rangeland metadata,4
4,merged_land_human_pressures,Shows areas where there is high anthropogenic ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Land human pressures metadata,5
...,...,...,...,...,...,...,...
81,mammals-rarity-1km,Each cell in this view measures 1 km x 1 km. W...,Map of Life and supporting datasets.,TRUE,FALSE,Mammals rarity metadata,82
82,summer-birds-richness-1km,Breeding ranges of migratory birds. Each cell ...,Map of Life and supporting datasets.,TRUE,FALSE,Summer birds richness metadata,83
83,summer-birds-rarity-1km,Breeding ranges of migratory birds. Each cell ...,Map of Life and supporting datasets.,TRUE,FALSE,Summer birds rarity metadata,84
84,winter-birds-richness-1km,Non-breeding ranges of migratory birds. Each c...,Map of Life and supporting datasets.,TRUE,FALSE,Winter birds richness metadata,85


In [12]:
## Overlap rows in production table

overlap_rows = pd.merge(left = prod_sdf[updatable_cols], right = stag_sdf[updatable_cols], how='inner',
                       on = 'layerSlug')
overlap_rows

,layerSlug,description_x,source_x,molLogo_x,hasAdditionalContent_x,title_x,ObjectId2_x,description_y,source_y,molLogo_y,hasAdditionalContent_y,title_y,ObjectId2_y
0,urban_human_pressures,Shows areas where the land is used by urban ac...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Urban pressures metadata,1,Shows areas where the land is used by urban ac...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Urban pressures metadata,1
1,irrigated_human_pressures,Shows areas where the land is used by irrigate...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Irrigated agriculture metadata,2,Shows areas where the land is used by irrigate...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Irrigated agriculture metadata,2
2,rainfed_human_pressures,Shows areas where the land is used by rainfed ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rainfed agriculture metadata,3,Shows areas where the land is used by rainfed ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rainfed agriculture metadata,3
3,rangeland_human_pressures,Shows areas where the land is used by rangelan...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rangeland metadata,4,Shows areas where the land is used by rangelan...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Rangeland metadata,4
4,merged_land_human_pressures,Shows areas where there is high anthropogenic ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Land human pressures metadata,5,Shows areas where there is high anthropogenic ...,"(1) [Ellis, Erle C., et al., 2010](https://onl...",FALSE,FALSE,Land human pressures metadata,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,butterflies-richness,Species richness at the country level for butt...,"(1) [Pinkert S. et al., 2022](https://onlineli...",TRUE,FALSE,Butterflies richness by country metadata,75,Species richness at the country level for butt...,"(1) [Pinkert S. et al., 2022](https://onlineli...",TRUE,FALSE,Butterflies richness by country metadata,75
75,odonates-richness,Species richness at the country level for odon...,"(1) Sandall, E.L., Pinkert S., & Jetz, W. (20...",TRUE,FALSE,Odonates richness by country metadata,76,Species richness at the country level for odon...,"(1) Sandall, E.L., Pinkert S., & Jetz, W. (20...",TRUE,FALSE,Odonates richness by country metadata,76
76,sapindales-richness,Species richness at the country level for sapi...,(1) [Plants of the World Online (2019). Facili...,TRUE,FALSE,Sapindales richness by country metadata,77,Species richness at the country level for sapi...,(1) [Plants of the World Online (2019). Facili...,TRUE,FALSE,Sapindales richness by country metadata,77
77,butterflies-richness-1km,Each cell in this view measures 1 km x 1 km. W...,Map of Life and supporting datasets.,TRUE,FALSE,Butterflies richness metadata,78,Each cell in this view measures 1 km x 1 km. W...,Map of Life and supporting datasets.,TRUE,FALSE,Butterflies richness metadata,78


In [13]:
features_for_update = [] #list containing corrected features
all_features = prod_fset.features

In [14]:
## Identify fields to be updated

for layerSlug in overlap_rows['layerSlug']:
    # get the feature to be updated
    original_feature = [f for f in all_features if f.attributes['layerSlug'] == layerSlug][0]
    feature_to_be_updated = deepcopy(original_feature)
    
    
    # get the matching row from dev
    matching_row = stag_sdf[updatable_cols].where(stag_sdf[updatable_cols].layerSlug == layerSlug).dropna()
    
    feature_to_be_updated.attributes['description'] = matching_row['description'].values[0]
    feature_to_be_updated.attributes['source'] = matching_row['source'].values[0]
    feature_to_be_updated.attributes['molLogo'] = matching_row['molLogo'].values[0]
    feature_to_be_updated.attributes['hasAdditionalContent'] = matching_row['hasAdditionalContent'].values[0]
    feature_to_be_updated.attributes['title'] = matching_row['title'].values[0]
    feature_to_be_updated.attributes['ObjectId2'] = int(matching_row['ObjectId2'])
    
    # add this to the list of features to be updated
    features_for_update.append(feature_to_be_updated)


In [15]:
# Update fields
flayer_prod.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 1,
   'uniqueId': 1,
   'globalId': None,
   'success': True},
  {'objectId': 2, 'uniqueId': 2, 'globalId': None, 'success': True},
  {'objectId': 3, 'uniqueId': 3, 'globalId': None, 'success': True},
  {'objectId': 4, 'uniqueId': 4, 'globalId': None, 'success': True},
  {'objectId': 5, 'uniqueId': 5, 'globalId': None, 'success': True},
  {'objectId': 6, 'uniqueId': 6, 'globalId': None, 'success': True},
  {'objectId': 7, 'uniqueId': 7, 'globalId': None, 'success': True},
  {'objectId': 8, 'uniqueId': 8, 'globalId': None, 'success': True},
  {'objectId': 9, 'uniqueId': 9, 'globalId': None, 'success': True},
  {'objectId': 10, 'uniqueId': 10, 'globalId': None, 'success': True},
  {'objectId': 11, 'uniqueId': 11, 'globalId': None, 'success': True},
  {'objectId': 12, 'uniqueId': 12, 'globalId': None, 'success': True},
  {'objectId': 13, 'uniqueId': 13, 'globalId': None, 'success': True},
  {'objectId': 14, 'uniqueId': 14, 'globalId': Non

#### 2. Identify new rows

In [16]:
new_rows = stag_sdf[~stag_sdf['layerSlug'].isin(overlap_rows['layerSlug'])]

In [17]:
new_rows

,layerSlug,description,source,molLogo,hasAdditionalContent,title,ObjectId,GlobalID,ObjectId2,ObjectId3
79,carbon-layer-land,Irrecoverable carbon refers to the vast stores...,"[Noon et al., 2021](https://www.nature.com/art...",FALSE,FALSE,Irrecoverable Land Carbon,None,None,80,80
80,mammals-richness-1km,Each cell in this view measures 1 km x 1 km. W...,Map of Life and supporting datasets.,TRUE,FALSE,Mammals richness metadata,None,None,81,81
81,mammals-rarity-1km,Each cell in this view measures 1 km x 1 km. W...,Map of Life and supporting datasets.,TRUE,FALSE,Mammals rarity metadata,None,None,82,82
82,summer-birds-richness-1km,Breeding ranges of migratory birds. Each cell ...,Map of Life and supporting datasets.,TRUE,FALSE,Summer birds richness metadata,None,None,83,83
83,summer-birds-rarity-1km,Breeding ranges of migratory birds. Each cell ...,Map of Life and supporting datasets.,TRUE,FALSE,Summer birds rarity metadata,None,None,84,84
84,winter-birds-richness-1km,Non-breeding ranges of migratory birds. Each c...,Map of Life and supporting datasets.,TRUE,FALSE,Winter birds richness metadata,None,None,85,85
85,winter-birds-rarity-1km,Non-breeding ranges of migratory birds. Each c...,Map of Life and supporting datasets.,TRUE,FALSE,Winter birds rarity metadata,None,None,86,86


In [18]:
## Identify new rows that need to be updated

features_to_be_added = []

# get a template feature object
template_feature = deepcopy(features_for_update[0])

# loop through each row and add to the list of features to be added
for row in new_rows.iterrows():
    new_feature = deepcopy(template_feature)
    
    #print
    print("Creating " + row[1]['layerSlug'])
    
    # assign the updated values 
    new_feature.attributes['layerSlug'] = row[1]['layerSlug']
    new_feature.attributes['description'] = row[1]['description']
    new_feature.attributes['source'] = row[1]['source']
    new_feature.attributes['molLogo'] = row[1]['molLogo']
    new_feature.attributes['hasAdditionalContent'] = row[1]['hasAdditionalContent']
    new_feature.attributes['title'] = row[1]['title']
    new_feature.attributes['ObjectId2'] = int(row[1]['ObjectId2'])
    
    
    # add this to the list of features to be updated
    features_to_be_added.append(new_feature)

Creating carbon-layer-land 
Creating mammals-richness-1km
Creating mammals-rarity-1km
Creating summer-birds-richness-1km
Creating summer-birds-rarity-1km
Creating winter-birds-richness-1km
Creating winter-birds-rarity-1km


#### Update `metadata_prod` table

In [19]:
flayer_prod.edit_features(adds = features_to_be_added)

{'addResults': [{'objectId': 80,
   'uniqueId': 80,
   'globalId': None,
   'success': True},
  {'objectId': 81, 'uniqueId': 81, 'globalId': None, 'success': True},
  {'objectId': 82, 'uniqueId': 82, 'globalId': None, 'success': True},
  {'objectId': 83, 'uniqueId': 83, 'globalId': None, 'success': True},
  {'objectId': 84, 'uniqueId': 84, 'globalId': None, 'success': True},
  {'objectId': 85, 'uniqueId': 85, 'globalId': None, 'success': True},
  {'objectId': 86, 'uniqueId': 86, 'globalId': None, 'success': True}],
 'updateResults': [],
 'deleteResults': []}

In [20]:
features_to_be_added

[{"attributes": {"layerSlug": "carbon-layer-land ", "description": "Irrecoverable carbon\u202frefers to the vast stores of carbon in nature that are vulnerable to release from human activity and, if lost, could not be restored by 2050 \u2014 when the world must reach net-zero emissions to avoid the worst impacts of climate change. This layer shows tonnes of irrecoverable carbon per hectare in both biomass and soils.", "source": "[Noon et al., 2021](https://www.nature.com/articles/s41893-021-00803-6)", "molLogo": "FALSE", "hasAdditionalContent": "FALSE", "title": "Irrecoverable Land Carbon", "ObjectId": null, "GlobalID": "136e5a63-a0e7-4100-9741-2650b353e36e", "ObjectId2": 80}},
 {"attributes": {"layerSlug": "mammals-richness-1km", "description": "Each cell in this view measures 1 km x 1 km. Within this region, species distributions were mapped at this detail using sophisticated modeling incorporating a variety of data types to provide this unprecedented resolution. The Half-Earth Proje